In [1]:

# Author: Sara_Fraija
import os
import pandas as pd
import sys
# Configuración de rutas
PATH_BASE='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/'
GRBsINFO = PATH_BASE+f'data/ULs/config/GRBs_list.csv'
bin_size = 0.6  ##PSF
PATH_GRBs_Healpix=PATH_BASE+f'data/healpix_maps_2.07_ER/'
PATH_GENERAL = PATH_BASE+f'data/ULs/config/Coordinates/{bin_size}/'
energy_ranges = PATH_BASE+f'data/ULs/config/Energy_ranges_2.07.csv'
GRBINFO_folder = PATH_BASE+f'data/ULs/config/'
OUTPUT_folder = PATH_BASE+f'data/ULs/files/{bin_size}/PSF_{bin_size}/'
PATH_SH = PATH_BASE+f'data/ULs/codes/'
RECALCULATED = PATH_BASE+f'data/corrected/'
File_1st_transit = PATH_BASE+f'data/ULs/config/Coordinates/0.6/Coordinates_with_Max_Sig_1_00.{bin_size}.csv'
File_2nd_transit = PATH_BASE+f'data/ULs/config/Coordinates/0.6/Coordinates_with_Max_Sig_2_00.{bin_size}.csv'
EBL_list = ['Franceschini08', 'Gilmore12Fiducial']    
LABEL_UL_CODE='ER'
LABELS = ['1', '2']
#############################################################
#############################################################





# Leer el archivo CSV
GRBs = pd.read_csv(GRBsINFO)
PATH_CODES = os.path.join(PATH_GENERAL, 'scripts')
os.makedirs(OUTPUT_folder, exist_ok=True)
os.makedirs(PATH_GENERAL,exist_ok=True)
window_size = bin_size * 2
def generate_bash_script(grb, ra, dec, bin_size, path):
    

    """Genera un script Bash para un GRB específico."""
    PATH_GRBs = os.path.join(path, grb)
    hist_path = os.path.join(PATH_GENERAL, 'SIG_ALL_GRBs')
    os.makedirs(hist_path, exist_ok=True)
    bin_size = max(bin_size, GRBs.loc[GRBs['Name'] == grb, 'Error_Radius'].values[0])

    script = f"""#!/usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_GRBs}/{grb}/maps/codes/HIST.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL{grb}
# Author: Sara_Fraija

source /data/disk01/home/smfraijac/.bashrc
hawc_aerie

RA={ra}
DEC={dec}
Bin_size={bin_size}
Hist_path={hist_path}
Window_size={window_size}
for Label in {LABELS[0]} {LABELS[1]}; do
    GRB={grb}_$Label
    output_file={hist_path}/{grb}_PSF_{bin_size}_Hist$Label.txt
    max_output_file={PATH_GENERAL}/MaxSigHist_{bin_size}_$Label.txt
    rm -f "$output_file"

    transit_file={PATH_GRBs}/transit_$Label/{grb}_transit_$Label.fits.gz
    # PL=$(makeSignificanceHistogram.py "$transit_file" --ra "$RA" --dec "$DEC" --includeArea --batch --binsize "$Bin_size" -o "$Hist_path/$GRB.png")
    PL=$(plotMercator.py "$transit_file"  --magma --cross  --circle $RA $DEC $Bin_size --circleproperty white 2 1 --origin $RA $DEC $Window_size $Window_size -o "$Hist_path/$GRB.png")
    echo -e "$PL" > "$output_file"

    Maxi=$(grep "Max" "$output_file")
    echo -e "{grb},$Label,$Maxi" >> "$max_output_file"
done
"""
    return script

def create_script_file(content, grb, bin_size):
    """Crea y guarda el script Bash en un archivo."""
    file_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    if os.path.exists(file_path):
        os.remove(file_path)
    with open(file_path, 'w') as file:
        file.write(content)

def generate_scripts_for_all_grbs(bin_size, path):
    """Genera scripts para todos los GRBs en el archivo CSV."""
    for _, row in GRBs.iterrows():
        grb, ra, dec = row['Name'], row['Ra'], row['Dec']
        script_content = generate_bash_script(grb, ra, dec, bin_size, path)
        create_script_file(script_content, grb, bin_size)

def generate_batch_sender(bin_size):
    """Genera un script para ejecutar todos los scripts generados."""
    batch_file_path = os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')
    os.makedirs(os.path.dirname(batch_file_path), exist_ok=True)
    if os.path.exists(batch_file_path):
        os.remove(batch_file_path)
    with open(batch_file_path, 'w') as batch_file:
        batch_file.write(f'rm -f {PATH_GENERAL}/MaxSig_{bin_size}_*.txt\n')
        for grb in GRBs['Name']:
            script_path = os.path.join(PATH_CODES, f'MaxSig{grb}_{bin_size}.sh')
            batch_file.write(f'sh {script_path}\n')

# Generar scripts para bin_size = 0.6

generate_scripts_for_all_grbs(bin_size, PATH_GRBs_Healpix)
generate_batch_sender(bin_size)
import subprocess
subprocess.run(["bash", os.path.join(PATH_CODES, f'BatchSenderMax_{bin_size}.sh')])

Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line


KeyboardInterrupt: 

In [ ]:

import pandas as pd
import numpy as np
path = GRBsINFO
coordi = pd.read_csv(path)
# Lista para almacenar los DataFrames
transit = 1
# Ruta del archivo y procesamiento
for transit in [1, 2]:
    dfs = []    
    for psf in [bin_size, 0.7]:
        path = PATH_GENERAL+f'/MaxSigHist_{psf}_{transit}.txt'
        # Leer el archivo de texto
        with open(path, 'r') as file:
            lines = file.readlines()
        
        # Procesar las líneas para extraer los datos
        data = []
        for line in lines:
            # Eliminar espacios en blanco al inicio y final de la línea
            line = ' '.join(line.split())
            line = line.replace('Max:', '').replace(')', '').replace('(', '').replace(' ', ',').replace(',,', ',')
            # Reemplazar y dividir la línea
            line = line.split(',')
            # Agregar a la lista de datos
            data.append(line)
        
        # # Convertir a un array de numpy y transponer
        data = np.array(data)
        # # Crear un DataFrame
        columns = ['Name', 'transit', 'Significance', 'RA', 'DEC']
        # print(data)
        # print(len(data[0]),data)
        if len(data[0])==len(columns):
            df = pd.DataFrame(data,columns=columns)        
            # Agregar una columna para identificar el valor de psf
            df['PSF'] = psf
            df.drop_duplicates(inplace=True)
            # Agregar el DataFrame a la lista
            dfs.append(df)
    # Combinar todos los DataFrames en uno solo
    combined_df = pd.concat(dfs, ignore_index=True).drop_duplicates()
    C=combined_df.join(coordi.set_index('Name'),on='Name')
    C.to_csv(f'../config/Coordinates_with_Max_Sig_{transit}_0{bin_size}.csv')
    # Mostrar el DataFrame combinado
    print(C)

            Name transit Significance      RA     DEC  PSF       Ra      Dec  \
0   GRB150110923       1         2.15  290.00   32.89  0.6  289.360  32.5220   
1   GRB150922234       1         2.31  294.43   -5.27  0.6  294.381  -5.4860   
2   GRB160726065       1         0.93   98.26   -6.17  0.6   98.821  -6.6440   
3   GRB170325331       1         2.03  128.10   19.95  0.6  127.483  20.5260   
4   GRB180715755       1         0.59  235.20   -1.31  0.6  235.069  -0.9250   
5   GRB180718082       1         1.58  335.65    2.87  0.6  336.019   2.7900   
6   GRB181125371       1         1.57  267.80   -3.17  0.6  268.413  -2.6130   
7   GRB190427190       1         2.55  280.06   40.13  0.6  280.213  40.3220   
8   GRB201008443       1         2.30  161.43   46.33  0.6  161.859  46.1019   
9   GRB201221963       1         1.26  171.25   41.51  0.6  171.059  42.1432   
10  GRB210323918       1         2.32  317.55   25.90  0.6  317.946  25.3699   
11  GRB170817529       1         2.72  1

/tmp/ipykernel_1479288/2194796600.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


In [ ]:
File_1st_transit = PATH_BASE+f'data/ULs/config/Coordinates/0.6/Coordinates_with_Max_Sig_1_0{bin_size}.csv'
File_2nd_transit = PATH_BASE+f'data/ULs/config/Coordinates/0.6/Coordinates_with_Max_Sig_2_0{bin_size}.csv'
import pandas as pd
import subprocess
import os
spectral_index=2.5
spectral_index_2=2.07
# !/usr/bin/env python3
# AUTHOR: Sara Fraija
# This script allows generating bash files to compute upper limits for each GRB. 
# You can change the GRB list, energy ranges, and EBL model.

# ------------------------------------------------------------------------
# PATHS & FILENAMES
# ------------------------------------------------------------------------

# Leer Energy_ranges.csv con nombres de columnas correctos
Energy_ranges = pd.read_csv(energy_ranges, names=['Name', 'Emin', 'Emax'])
Energy_ranges = Energy_ranges.set_index('Name')
Energy_ranges.sort_values(by='Name')
# ------------------------------------------------------------------------
# Function to generate bash script
# ------------------------------------------------------------------------
def script(GRBINFO, transit, PATH_SH, OUTPUT_folder, RECALCULATED, EBL, PSF):
    OUTPUT_FILE = f"{OUTPUT_folder}UpperLimit_{transit}_{EBL}.csv"

    # Leer archivo CSV con GRB information
    Data = pd.read_csv(GRBINFO)
    Data.sort_values('Name')
    Data.drop_duplicates(inplace=True)
    # Verificar si la columna "Name" existe en el CSV
    if "Name" not in Data.columns:
        print(f"Error: La columna 'Name' no está en {GRBINFO}. Verifica el formato del archivo.")
        print(f"Columnas disponibles: {Data.columns}")
        return

    # Usar "Name" como índice
    Data = Data.set_index("Name")

    # Asignar valores de Emin y Emax si existen en Energy_ranges
    Data['Emin'] = Data.index.map(Energy_ranges['Emin'])
    Data['Emax'] = Data.index.map(Energy_ranges['Emax'])
    # Verificar si hay valores NaN en Emin o Emax
    if Data[['Emin', 'Emax']].isna().any().any():
        print("Error: Algunos GRBs no tienen valores de Emin o Emax. Verifica Energy_ranges.csv.")
        print(Data[['Emin', 'Emax']].isna().sum())
        return

    line = ""
    for grb in Data.index:  
        row = Data.loc[grb]
        RA = row['RA']
        Dec = row['DEC']
        z = row['z']
        print(RA)
        index = spectral_index_2 if grb == 'GRB170817529' else spectral_index

        Emin = row['Emin']
        Emax = row['Emax']
        line += f'PL="PowerLaw,1.0e-10,{index}"\n'
        line_1 = f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB0C1_N1024.fits.gz $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{1..10}}C{{0..1}}_N1024.fits.gz -b B0C1 B{{1..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
        line_2 = f"ULs=$(zebra-flux-norm-fit -i $dir/{grb}/transit_{transit}/{grb}_corrected_transit_{transit}_binB{{0..10}}C{{0..1}}_N1024.fits.gz -b B{{0..10}}C{{0..1}} --extrapolate-low-z --dr $DR51 --ra {RA} --dec {Dec} -s $PL --ebl {EBL},{z} --pivot 1 --minE {Emin} --maxE {Emax} -V 3)\n"
        line += line_1 if grb == 'GRB150110923' else line_2
        line += f'echo -e "{grb},${{ULs}}" >> $output_file\n'
        
    # Código base del script Bash
    code = f"""#! /usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_SH}/logs/UL.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL
# Author: Sara Fraija
source /data/disk01/home/smfraijac/.bashrc
hawc_aerie
output_file={OUTPUT_FILE}
DR51={PATH_BASE}/config/pass5.f_DR/zebra-100pct-FHit.root
dir={RECALCULATED}\n
output_file={OUTPUT_FILE}\n
rm -f output_file
"""
    # Guardar el script Bash
    file_path = f"{PATH_SH}UL{transit}_{EBL}_{bin_size}_{LABEL_UL_CODE}.sh"
    if os.path.exists(file_path):
        os.remove(file_path)
    with open(file_path, 'w') as bash_file:
        bash_file.write(code + line)
    print(f"Script generado: {file_path}")
# ------------------------------------------------------------------------
# Running script
# ------------------------------------------------------------------------

for EBL in EBL_list:
    script(File_1st_transit, transit=1, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF={bin_size})
    script(File_2nd_transit, transit=2, PATH_SH=PATH_SH, OUTPUT_folder=OUTPUT_folder, RECALCULATED=RECALCULATED, EBL=EBL, PSF={bin_size})
    p1 = subprocess.Popen(['bash', f"{PATH_SH}/UL1_{EBL}_{bin_size}_{LABEL_UL_CODE}.sh"])
    p2 = subprocess.Popen(['bash', f"{PATH_SH}/UL2_{EBL}_{bin_size}_{LABEL_UL_CODE}.sh"])


290.0
294.43
98.26
128.1
235.2
335.65
267.8
280.06
161.43
171.25
317.55
197.67
Name
GRB170403583    259.15
GRB170403583    258.71
Name: RA, dtype: float64
Name
GRB170403583    259.15
GRB170403583    258.71
Name: RA, dtype: float64
Script generado: /lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/UL1_Franceschini08_0.6_ER.sh
Name
GRB170403583    259.28
GRB170403583    259.98
Name: RA, dtype: float64
Name
GRB170403583    259.28
GRB170403583    259.98
Name: RA, dtype: float64
Script generado: /lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/UL2_Franceschini08_0.6_ER.sh
290.0
294.43
98.26
128.1
235.2
335.65
267.8
280.06
161.43
171.25
317.55
197.67
Name
GRB170403583    259.15
GRB170403583    258.71
Name: RA, dtype: float64
Name
GRB170403583    259.15
GRB170403583    258.71
Name: RA, dtype: float64
Script generado: /lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/UL1_Gilmore12Fiducial_0.6_ER.sh
Name
GRB170403583    259.28
GRB1704035

Activate HAWC enviroment
Activate HAWC enviroment
Activate HAWC enviroment
Activate HAWC enviroment


>>> Configuration file found, will be ignored in favour of command line
>>> Configuration file found, will be ignored in favour of command line
>>> Configuration file found, will be ignored in favour of command line
>>> Configuration file found, will be ignored in favour of command line
ERROR [CommandLineConfigurator.cc, ParseCommandLine:155]: Command line error: the argument ('Name') for option '--ra' is invalid
Allowed options:

Generic options:
  -h [ --help ]                Write this message
  -v [ --version ]             Print Aerie version info
  -V [ --verbosity ] arg (=2)  Set the verbosity of the logging system: 
                               0=trace, 1=debug, 2=info, 3=warn, 4=error, 
                               5=fatal
  -x [ --fpexcept ] arg (=0)   Enable floating-point exceptions for debugging: 
                               1=invalid-arg, 4=div-by-zero, 8=overflow, 13=all
  --timelog                    Include current UT time in AERIE logs.

Configuration:
  -i [ --

In [ ]:
f"{PATH_SH}/UL1_{EBL}_{bin_size}_{LABEL_UL_CODE}.sh"

'/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes//UL1_Gilmore12Fiducial_0.6_ER.sh'